A faire : 
- parcourir toutes les dates
- cliquer sur un jour 
- cliquer sur un tournoi
- aller dans l'onglet "grille" (s'il y en a) et lire les informations : 
- mettre dans un tableau, pour chaque joueur le nom du tournoi, le classement du joueur dans le tournoi, contre qui il a joué (avec le score) 

sous la forme d'un dictionnaire {'nom du joueur' : untel, 'elo': 2000, 'tournois' : [{'nom' : Tournoi du pin, 'date' : 15/04/2022, 'classement' : 13}, {'nom' : Tournoi du chêne, 'date' : 15/02/2021, 'classement' : 8}] , 'adversaires' : [{'nom' : untel, 'elo' : 1900, 'score' : perdu, 'couleur' : noirs, 'date' : 15/09/2020}, {'nom' : untel, 'elo' : 1500, 'score' : perdu, 'couleur' : noirs, 'tournoi' : Nationale 3, 'date' : 13/08/2021}]}

#indication : =/- : gagné, nul, perdu , 4 : chiffre de l’adversaire, B/N : blancs ou noirs 

In [1]:
import requests

#out = requests.get("http://echecs.asso.fr/FicheTournoi.aspx?Ref=55579")
#out = requests.get("http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/55579/55579&Action=Ga")
out = requests.get("http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/55605/55605&Action=Berger")
out.headers
print(out.content.decode())



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" >
<head id="ctl00_MasterHeader"><meta http-equiv="content-type" content="application/xhtml+xml; charset=utf-8" /><meta name="keywords" /><meta name="description" /><title>
	F&eacute;d&eacute;ration Fran&ccedil;aise des &Eacute;checs
</title><link type="text/css" rel="stylesheet" href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" /><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css" /><link type="text/css" rel="stylesheet" href="ffe.css?version=6" /><link type="text/css" rel="stylesheet" href="old.css?version=1" /><link rel="shortcut icon" type="image/x-icon" href="images/ffe16.ico" />
  <script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o)

In [2]:
#RECUPERATION DES DONNES POUR UNE GRILLE BERGER

import requests
from bs4 import BeautifulSoup

def recup_berger(number):#number=numéro d'une grille berger
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Berger"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    #récupération des joueurs
    j=[]
    for x in soup.find_all('td'):
        if x.has_attr("class") and x['class'][0] == "papi_border_l" :
            j.append(x.string.lower())
    joueur=j[1:]
    
    #récupération des elo
    e=[]
    i=0
    for x in soup.find_all('tr'):
        if x.has_attr("class") and x['class'][0] == "papi_liste_c":
            for k,y in enumerate(x.find_all('td')):
                if (k-3)%15==0 and i<len(joueur)+1:
                    e.append(y.string)
                    i+=1
    elo=dict()
    for i in range(len(joueur)):
        elo[joueur[i]]=e[1:][i]
    
    #récupération des scores relatifs sous forme de dictionnaire
    dic=dict()
    for i in range(len(joueur)):
        p=[]
        for x in soup.find_all('tr'):
            if x.has_attr("class") and x['class'][0] == "papi_liste_c":
                for k,y in enumerate(x.find_all('td')):
                    if (k-4-i)%15==0:
                        p.append(y.string)
        perf_colonne=p[1:len(joueur)+1]
        dic[joueur[i]]=perf_colonne
        #avant on a récupéré tous les modulos 15, ie on a les scores en colonne, on doit les présenter en ligne   
    dictionnaire=dict()
    for i in range(8):
        if len(joueur)>=8:
            dico_joueur=dictionnaire[joueur[i]]= dict()
            for j in range(8):
                if j!=i:
                    dico_joueur[joueur[j]]=dic[joueur[j]][i]
    return(joueur,elo,dictionnaire)       
     

recup_berger(55605)

(['joachim martellier',
  'sonia oupindrin',
  'rayen ben ayed',
  'melaine tryhoen',
  'julien holveck',
  'roger baltes',
  'freddy halour',
  'claudine doublecourt'],
 {'joachim martellier': '1101\xa0F',
  'sonia oupindrin': '1018\xa0F',
  'rayen ben ayed': '1054\xa0F',
  'melaine tryhoen': '1128\xa0F',
  'julien holveck': '1170\xa0F',
  'roger baltes': '1120\xa0N',
  'freddy halour': '1083\xa0F',
  'claudine doublecourt': '1098\xa0F'},
 {'joachim martellier': {'sonia oupindrin': '½',
   'rayen ben ayed': '1',
   'melaine tryhoen': '1',
   'julien holveck': '½',
   'roger baltes': '1',
   'freddy halour': '1',
   'claudine doublecourt': '1'},
  'sonia oupindrin': {'joachim martellier': '½',
   'rayen ben ayed': '0',
   'melaine tryhoen': '½',
   'julien holveck': '1',
   'roger baltes': '1',
   'freddy halour': '1',
   'claudine doublecourt': '1'},
  'rayen ben ayed': {'joachim martellier': '0',
   'sonia oupindrin': '1',
   'melaine tryhoen': '½',
   'julien holveck': '0',
   'roge

In [3]:
base = dict()

number = 55579

#checker si la page est américaine 
def american(number):
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('span'):
        if x.string == "Désolé, le fichier n'existe pas...":
            print("berger")
            return False
        else :
            print("américain")
            return True 


#pour avoir la date, le nom, la cadence, et les rondes du tournoi
def infos_tournoi(number):  
    infos = dict()
    link_infos = f"http://echecs.asso.fr/FicheTournoi.aspx?Ref={number}"
    soup_infos = BeautifulSoup(requests.get(link_infos).text, 'html.parser')
    for x in soup_infos.find_all('span'):
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelDates":
            infos["date"] = x.string   
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNom":
            infos["nom"] = x.string  
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNbrRondes":
            infos["nb_rondes"] = x.string
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelCadence":
            infos["cadence"] = x.string 
    return infos



#dans la boucle, si la date c'est demain, on arrête la boucle 

from datetime import datetime

months = {"janvier":1, "février":2, "mars":3, "avril":4,
       "mai":5, "juin":6, "juillet":7, "août":8,
       "septembre":9, "octobre":10, "novembre":11, "décembre":12}

def stop_boucle(date): #fonction qui renvoie true si on doit arrêter la boucle
    liste_date = date.strip('-').split(' ')
    if liste_date[0]=='' and liste_date[1]=='-' and liste_date[2]=='':
        return False 
    for k in range(len(liste_date)):
        if k%5==1:
            jour = liste_date[k]
        if k%5==2 :
            mois = months[liste_date[k]]
        if k%5==3:
            année = liste_date[k]
    jour_now, mois_now, année_now = datetime.today().day, datetime.today().month, datetime.today().year
    if int(année)>int(année_now):
        return True
    elif int(année)==int(année_now):
        if int(mois)>int(mois_now):
            return True
        elif int(mois)==int(mois_now):
            if int(jour)>int(jour_now):
                return True
    return False
        
#print(stop_boucle('mercredi 23 juin 2022 - samedi 8 juillet 2022'))


In [4]:
#marche à suivre
#faire une boucle sur les numéros de match
#while la date est encore à jour
#tester si la page est américaine ou berger 
#collecter les infos du tournoi et les mettre dans la table tournoi
#récupérer la liste des joueurs
#pour chaque player, tester s'il est déjà dans la base 
#récupérer les matchs et les mettre dans la base de donnée 

In [5]:
import sqlite3

#création de la base de données 
con = sqlite3.connect(":memory:")
con.executescript("""
create table player (id integer not NULL primary key autoincrement, name text not NULL); 
create table tournois (id integer not NULL primary key autoincrement, name text not NULL, date text not NULL, rondes integer, cadence string);
create table match (id integer primary key autoincrement, tournoi integer, black_player integer, elo_black string, white_player integer, elo_white string, winner integer);""")

#con.execute("insert into player(name) values (?)", ("philippe marchal",))
#con.execute("insert into player(id, name) values (?, ?)", (0, "pas d'adversaire"))

#for row in con.execute("select name from player where name = 'philippea marchal'"):
    #print(row)


    
#print(list(con.execute("select name from player where name = 'philippe marchal'")))[0]

#winner : 0 pour les noirs, 3 pour les noirs au temps, 1 pour les blancs, 4 pour les blancs au temps, 2 pour ex aequo

In [6]:
#savoir lire ce genre de grille américaine
link = "http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/30005/30005&Action=Ga"
soup = BeautifulSoup(requests.get(link).text, 'html.parser')


print(soup)



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_MasterHeader"><meta content="application/xhtml+xml; charset=utf-8" http-equiv="content-type"/><meta name="keywords"/><meta name="description"/><title>
	Fédération Française des Échecs
</title><link href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" rel="stylesheet" type="text/css"/><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css"/><link href="ffe.css?version=6" rel="stylesheet" type="text/css"/><link href="old.css?version=1" rel="stylesheet" type="text/css"/><link href="images/ffe16.ico" rel="shortcut icon" type="image/x-icon"/>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.a

In [7]:
def players_ga(number):
    indicateur = 0
    res = []
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('div'):
        if x.has_attr("class") and x['class'][0] == "papi_joueur_box":
            res.append(x.b.string)
    indicateur = 1
    if res == []:
        for x in soup.find_all('td') :
            if x.has_attr("class") and x["class"][0]=="papi_l":
                res.append(x.string)
                
        res = res[1:]
        indicateur = 2
              
    return (res, indicateur)


def match_ga(number):
    symboles = ['+', '-', "=", ">", "<"]
    players, ind = players_ga(number)
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    res = []

    if ind== 1:
        adversaires = []
        elos = []
        couleurs = []
        résultats = []
        for x in soup.find_all('div'):
            if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
                joueur = []
                elo = []
                couleur = []
                résultat = []
                for k,y in enumerate(x.find_all('td')):
                    if (k-16)%13==0:
                        joueur.append(y.string)
                    if (k-17)%13==0:
                        z=str(y.string).split("\xa0")
                        elo.append(z)
                    if (k-12)%13==0:
                        couleur.append(y.string)
                    if k%13==0:
                        résultat.append(y.string)
                adversaires.append(joueur) 
                elos.append(elo)
                couleurs.append(couleur)
                résultats.append(résultat)
    if ind == 2:
        adversaires = []
        elos = []
        couleurs = []
        résultats = []
        for j,x in enumerate(soup.find_all('tr')):
            if j>2:
                joueur = [players[j-3]]
                elo = []
                couleur = []
                résultat = []
                for k,y in enumerate(x.find_all('td')):
                    if y.has_attr("class") and y['class'][0] == "papi_r":
                        if k==3:
                            z=str(y.string).split("\xa0")
                            elo.append(z)
                        if y.string!=None and y.string[0] in symboles and len(y.string)>4:
                            joueur.append(players[int(y.string[3])-1])
                            couleur.append(y.string[4])
                            résultat.append(y.string[0])
                adversaires.append(joueur) 
                elos.append(elo)
                couleurs.append(couleur)
                résultats.append(résultat)

    déjà_vus = set()
    if ind==1:
        for k in range (len(adversaires)):
            for i in range(1,len(adversaires[k])):
                match = []
                try:
                    match.append(adversaires[k][0])
                    déjà_vus.add(adversaires[k][0])
                    match.append(adversaires[k][i])
                    match.append(elos[k][0])
                    match.append(elos[k][i])
                    match.append(couleurs[k][i-1])
                    match.append(résultats[k][i])
                    if adversaires[k][i] not in déjà_vus:
                        res.append(match)
                except IndexError:
                    pass
    if ind ==2:
        for k in range (len(adversaires)):
            for i in range(1,len(adversaires[k])):
                match = []
                match.append(adversaires[k][0])
                déjà_vus.add(adversaires[k][0])
                match.append(adversaires[k][i])
                match.append(elos[k][0])
                match.append(elos[players.index(adversaires[k][i])][0])
                match.append(couleurs[k][i-1])
                match.append(résultats[k][i-1])
                if adversaires[k][i] not in déjà_vus:
                    res.append(match)
    return res
                      
match_ga(30012)   
#match_ga(55579)

[]

In [8]:
#on insère les infos d'un match américain dans les tables

def insere_ga(number):
    #inserer le tournoi
    date_tournoi = infos_tournoi(number)["date"]
    nom_tournoi = infos_tournoi(number)["nom"]
    nb_rondes_tournoi = infos_tournoi(number)["nb_rondes"]
    cadence_tournoi = infos_tournoi(number)["cadence"]
    con.execute("insert into tournois(id, name, date, rondes, cadence) values (?,?,?,?,?)", (number, nom_tournoi, date_tournoi, nb_rondes_tournoi, cadence_tournoi))
    
    #inserer les joueurs du tournoi
    players, ind = players_ga(number)
    for player in players:
        p = player.lower()
        recherche = list(con.execute("select name from player where name = ?", (p,)))
        if recherche == []:
            con.execute("insert into player(name) values (?)", (player.lower(),))
    
    #inserer les matchs du tournoi
    for element in match_ga(number):
        print(element)
        #id_tournoi = list(con.execute("select id from tournois where name = ?", (nom_tournoi,)))[0][0]
        if element[1]!= 'EXEMPT' and element[1]!=None and element[0]!='EXEMPT' and element[0]!=None:
            p_1 = list(con.execute("select id from player where name = ?", (element[0].lower(),)))
            p_2 = list(con.execute("select id from player where name = ?", (element[1].lower(),)))
            if p_1!=[] and p_2!=[]:
                player_1 = list(con.execute("select id from player where name = ?", (element[0].lower(),)))[0][0]
                player_2 = list(con.execute("select id from player where name = ?", (element[1].lower(),)))[0][0]
                if len(element[2])==2 :
                    elo_1 = element[2][0]+element[2][1]
                else :
                    elo_1 = 0
                if len(element[3])==2:
                    elo_2 = element[3][0]+element[3][1]
                else :
                    elo_2 = 0
                if element[4] == 'N':
                    black_player = player_1
                    elo_black = elo_1
                    white_player = player_2
                    elo_white = elo_2
                    if element[5]=='1' or element[5]=='+' :
                        winner = 0
                    elif element[5]=='½' or element[5]=="=":
                        winner = 2
                    elif element[5]=='0' or element[5]=='-':
                        winner = 1 
                    elif element[5]=='>':
                        winner = 3
                    elif element[5]=='<':
                        winner = 4
                else :
                    white_player = player_1
                    elo_white = elo_1
                    black_player = player_2
                    elo_black = elo_2
                    if element[5]=='1' or element[5]=='+' :
                        winner = 1
                    elif element[5]=='½' or element[5]=="=":
                        winner = 2
                    elif element[5]=='0' or element[5]=='-':
                        winner = 0
                    elif element[5]=='>':
                        winner = 4
                    elif element[5]=='<':
                        winner = 3
                con.execute("insert into match(tournoi, black_player, elo_black, white_player, elo_white, winner) values (?,?,?,?,?,?)", (number, black_player, elo_black, white_player, elo_white, winner))


In [9]:
#insertion pour berger 
#insertion de toutes les données relatives à un tournoi de type berger

def insertion_berger(number):#number=numéro du tournoi
    #insertion des joueurs du tournoi
    for player in recup_berger(number)[0]:
        recherche = list(con.execute("select name from player where name = ?", (player.lower(),)))
        if player not in recherche:
            con.execute("insert into player(name) values (?)",(player.lower(),))
    #insertion dans la table tournois        
    con.execute("insert or replace into tournois(id, name, date, rondes, cadence) values(?,?,?,?,?)",(number, infos_tournoi(number)["nom"],infos_tournoi(number)["date"],infos_tournoi(number)["nb_rondes"],infos_tournoi(number)['cadence'], ))   
    #insertion des matchs d'un tournoi
    i=0
    for player in recup_berger(number)[0]:
        dicomatch=recup_berger(number)[2]
        dicojoueur=dicomatch[player]
        joueurrestant=list(dicojoueur.keys())[i:]
        for adversaire in joueurrestant:
            black_id = list(con.execute("select id from player where name = ?", (player.lower(),)))[0][0]
            white_id = list(con.execute("select id from player where name = ?", (adversaire.lower(),)))[0][0]
            elo_black=recup_berger(number)[1][player]
            elo_white=recup_berger(number)[1][adversaire]
            if dicojoueur[adversaire]=='1':
                winner=0
            if dicojoueur[adversaire]=='0':
                winner=1
            else:
                winner=2
            con.execute("insert or replace into match(tournoi,black_player,elo_black,white_player,elo_white,winner) values(?,?,?,?,?,?)",(number,black_id,elo_black,white_id,elo_white,winner,))    
        i+=1

In [10]:
for row in con.execute("select * from match"):
    print(row)
    
for row in con.execute("select * from player"):
    print(row)
    
for row in con.execute("select * from tournois"):
    print(row)

In [76]:
#cellule à exécuter pour vider la base de donnée
con.execute("delete from player");
con.execute("delete from sqlite_sequence where name='player'");
for row in con.execute("select * from player"):
    print(row)
    
con.execute("delete from match");
con.execute("delete from sqlite_sequence where name='match'");
for row in con.execute("select * from match"):
    print(row)
    
con.execute("delete from tournois");
con.execute("delete from sqlite_sequence where name='tournois'");
for row in con.execute("select * from tournois"):
    print(row)

In [46]:
#29018
#30001
#28816
#28718, 28517 (cadence)
#split les elo
#28613
#28411 les joueurs ne se mettent pas dans la table ni les match
#27918

In [82]:
number = 23410
while number<23420 and not stop_boucle(infos_tournoi(number)['date']):
    number+=1
    print(number)
    if american(number):
        insere_ga(number)
    else :
        insertion_berger(number)
        
        
for row in con.execute("select * from match"):
    print(row)
    
for row in con.execute("select * from player"):
    print(row)
    
for row in con.execute("select * from tournois"):
    print(row)

23411
berger
23412
berger


KeyError: 'cadence'

In [5]:
#tentative de SQL

import sqlite3

CreateDataBase = sqlite3.connect('MyDataBase.db')

QueryCurs = CreateDataBase.cursor()

def CreateTable():
    QueryCurs.execute('''CREATE TABLE Joueurs
    (id INTEGER PRIMARY KEY, Nom TEXT,Elo TEXT, Nationalité TEXT)''')

def AddEntry(Nom,Elo,Nationalité):
    QueryCurs.execute('''INSERT INTO Joueurs (Nom,Elo,Nationalité)
    VALUES (?,?,?)''',(Nom,Elo,Nationalité))



AddEntry('Tom','105','FRance')


CreateDataBase.commit()

QueryCurs.execute('SELECT * FROM Joueurs')

for i in QueryCurs:
    print ("\n")
    for j in i:
        print (j)

QueryCurs.close()

OperationalError: no such table: Joueurs

In [9]:
#Commandes SQL

buffer = ""

print("Enter your SQL command to execute")
print("Enter a blank line to exit")

while True:
    line = input()
    if line =="":
        break
    buffer += line
    if sqlite3.complete_statement(buffer):
        try:
            buffer = buffer.strip()
            con.execute(buffer)
            for row in con.execute(buffer):
                print(row)
            
            if buffer.lstrip().upper().startswitch("SELECT"):
                print(cur.fetchall())
        except sqlite.Error as e :
            print("An error occured:", e.args[0] )
        buffer = ""


Enter your SQL command to execute
Enter a blank line to exit


KeyboardInterrupt: Interrupted by user